# Face Identification

1. MTCNN으로 이미지 내 얼굴을 인식하고 얼굴 부분만 오려낸다.

2. 얼굴 이미지를  "FaceNet: A Unified Embedding for Face Recognition and Clustering"을 통해 특징 벡터로 표현한다.

3. 이 특징 벡터로 SVM 분류기를 학습하여 Face Identification 모델을 만든다.

## Import Libraries

현재 이 코드가 적힌 파일 위치에 facenet_pytorch이라는 폴더가 있고, 이를 모듈처럼 불러와 사용하는데,

이것도 `__init__.py`라는 파일이 있어야 이렇게 모듈처럼 불러올 수 있고, 동일 폴더 내에 있어야 하는 등 여러 조건이 있다.

In [1]:
# 구글이 공개한 facenet이라는 AI 모델이 있는데, tensorflow로 작성된 것을 pytorch 형태로 공개하고 있다.
# https://github.com/timesler/facenet-pytorch
from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization, training

In [2]:
from __future__ import print_function, division
import os
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
import numpy as np
import pandas as pd

# 경고 메시지 무시하기
import warnings
warnings.filterwarnings("ignore")

## Import the Data

우리가 봤었던 MNIST의 경우, 데이터가 이미지인데 excel에 숫자로 적혀 있어서 pandas로 불러와서 이를 torch로 바꿔주었으나,

`featuresTrain = torch.from_numpy(features_train)`

`targetsTrain = torch.from_numpy(target_train).type(torch.LongTensor)`

`train = torch.utils.data.TensorDataset(featuresTrain,targetsTrain)`

여기서는 이미지 파일이기 때문에 torchvision.datasets이라는 내장된 모듈을 사용해 간편하게 불러온다.

***그리고 이미지를 align 해야 하는 과정이 필요하다.***

`dataset = datasets.ImageFolder(data_dir, transform=transforms.Resize((512, 512)))`

&nbsp;

***추가로, [Pytorch를 활용한 데이터 불러오는 과정에 대한 튜토리얼](https://tutorials.pytorch.kr/beginner/data_loading_tutorial.html)***

#### Define run parameters

The dataset should follow the VGGFace2/ImageNet-style directory layout. Modify `data_dir` to the location of the dataset on wish to finetune on.

보통 input data 위치 및 하이퍼파라미터를 설정하는 것인데, 하이퍼 파라미터란 조정하는 수치값을 의미한다.

epoch의 경우 같은 데이터로 학습을 몇 번 돌릴 것인지를 의미한다.

In [3]:
data_dir = './data/train'

workers = 0 if os.name == 'nt' else 4

#### Determine if an nvidia GPU is available

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cpu


#### Define MTCNN module

See `help(MTCNN)` for more details.

In [5]:
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709,
    device=device
)

## Pytorch를 사용하여 이미지를 불러올 때

1. 보통 `torchvision.datasets`로 데이터 객체를 선언하고, (`torchvision.datasets`은 `torch.utils.data.Dataset`의 서브 클래스이다)
2. `torch.utils.data.DataLoader`로 데이터 로드를 위한 객체를 선언하고,
3. 그 객체로 데이터를 로드한다.

데이터를 불러오는 과정에서 이러한 내장된 객체를 사용하는 이유는 여러 기능을 포함하기 때문이다.

`torchvision.datasets`의 경우,

- IMG_EXTENSIONS('.jpg', '.jpeg', '.png', '.ppm', '.bmp', '.pgm', '.tif', '.tiff', '.webp')에 대해서 처리 가능하고,
- 데이터를 PIL(Python Imaging Library) 객체로 바뀌어 python 내에서 통일적으로 효율적으로 처리된다.
- 또한 이미지를 쉽게 변형시킬 수 있는 transform 기능을 포함하고 있다.
- 마지막으로 datasets 내에는 유명한 데이터셋을 쉽게 불러오거나 다운로드 받을 수 있고 있고, (`torchvision.datasets.MNIST`)
- 폴더 경로만 주어지면 자동으로 index를 포함하여 데이터셋을 만들 수 있는 객체도 포함한다. (`torchvision.datasets.ImageFolder`)

`torch.utils.data.DataLoader`의 경우 아래의 5가지 대표적인 기능을 하며,

- map-style and iterable-style datasets, (map은 각 데이터에 대해서 같은 처리를 동시에 하는 방식이고, iterable을 순회해가며 처리)
- customizing data loading order,
- automatic batching,
- single- and multi-process data loading,
- automatic memory pinning.

`torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False, sampler=None, batch_sampler=None, num_workers=0, collate_fn=None, pin_memory=False, drop_last=False, timeout=0, worker_init_fn=None, multiprocessing_context=None)`

위를 보면, 다양한 parameter를 볼 수 있는데, parameter 변경으로 데이터를 로드할 때 여러 처리를 할 수 있다.

설명을 조금 하자면, Dataloader를 통해 batch로 혹은 non-batch로 데이터를 불러올 수 있는데, batch란 ***collating individual fetched data samples into batches***, 즉 개별 데이터를 batch 묶음 형태로 수집하는 것이다. Dataloader는 기본값으로 자동 batching을 지원한다.

개별 데이터의 index들을 사용하여 sample list로 묶이면, collate_fn 함수를 통해 batches로 수집(collate)된다.

```
for indices in batch_sampler:
    yield collate_fn([dataset[i] for i in indices])
```

중요한 것은,
The use of collate_fn is slightly different when automatic batching is enabled or disabled.

***When automatic batching is disabled***, collate_fn is called with each individual data sample, and the output is yielded from the data loader iterator. In this case, the default collate_fn simply converts NumPy arrays in PyTorch tensors.

***When automatic batching is enabled***, collate_fn is called with a list of data samples at each time. It is expected to collate the input samples into a batch for yielding from the data loader iterator. The rest of this section describes behavior of the default collate_fn in this case.

In [6]:
def collate_fn(x):
    return x[0]

dataset = datasets.ImageFolder(data_dir)
dataset.idx_to_class = {i:c for c, i in dataset.class_to_idx.items()}
loader = DataLoader(dataset, collate_fn=collate_fn, num_workers=workers)

In [8]:
for x, y in loader:
    print(x)
    print(y)
    raise # 멈추기 위해 일부러 오류 발생

# collate_fn에 따른 결과값, 순서대로 x, x[0], x[0][0]
# [(<PIL.Image.Image image mode=RGB size=490x489 at 0x1AD3C10C7F0>, 0)]
# (<PIL.Image.Image image mode=RGB size=490x489 at 0x1AD3C58BDD8>, 0)
# <PIL.Image.Image image mode=RGB size=490x489 at 0x1AD3C56AE48>

<PIL.Image.Image image mode=RGB size=490x489 at 0x1FCAEF98EB8>
0


RuntimeError: No active exception to reraise

In [9]:
aligned = []
names = []
# x는 이미지(<PIL.Image.Image image mode=RGB size=490x489 at 0x1AD3C42EC50>)
# y는 index
for x, y in loader:
    x_aligned, prob = mtcnn(x, return_prob=True)
    if x_aligned is not None:
        print('Face detected with probability: {:8f}'.format(prob))
        aligned.append(x_aligned)
        names.append(dataset.idx_to_class[y])

Face detected with probability: 0.998157
Face detected with probability: 0.999907
Face detected with probability: 0.999996
Face detected with probability: 0.998637
Face detected with probability: 0.992943
Face detected with probability: 0.999828
Face detected with probability: 0.999999
Face detected with probability: 0.999614
Face detected with probability: 0.999639
Face detected with probability: 0.999997
Face detected with probability: 0.999999
Face detected with probability: 0.999991
Face detected with probability: 0.999993
Face detected with probability: 0.999196
Face detected with probability: 0.999427
Face detected with probability: 0.999996
Face detected with probability: 0.999999
Face detected with probability: 0.995905
Face detected with probability: 0.999999
Face detected with probability: 0.999876
Face detected with probability: 0.999325
Face detected with probability: 0.999985
Face detected with probability: 0.998141
Face detected with probability: 0.999952
Face detected wi

In [10]:
resnet = InceptionResnetV1(pretrained='vggface2', classify=True).eval().to(device)

In [11]:
aligned = torch.stack(aligned).to(device)
embeddings = resnet(aligned).detach().cpu()

In [12]:
embeddings.size()

torch.Size([50, 8631])

In [13]:
dists = [[(e1 - e2).norm().item() for e2 in embeddings] for e1 in embeddings]
print(pd.DataFrame(dists, columns=names, index=names))

                   IU          IU          IU          IU          IU  \
IU           0.000000  107.810120  116.027176  108.889549  123.196365   
IU         107.810120    0.000000  102.467888   86.000793  131.691574   
IU         116.027176  102.467888    0.000000  100.256821  115.437256   
IU         108.889549   86.000793  100.256821    0.000000  130.040131   
IU         123.196365  131.691574  115.437256  130.040131    0.000000   
chanyoung  274.502075  241.052124  247.831802  263.818451  264.909210   
chanyoung  259.087280  236.850754  239.362000  255.538300  249.462402   
chanyoung  248.231018  228.814575  237.635712  240.089005  228.271973   
chanyoung  257.008362  232.050644  246.255280  244.457062  230.637451   
chanyoung  241.992126  237.529617  253.617783  238.546219  235.419189   
eunwoo     253.210632  252.720032  257.899109  260.373688  253.126205   
eunwoo     244.311539  243.839905  256.409576  246.702286  244.080246   
eunwoo     218.141876  220.076767  229.051682  234.

In [14]:
len(embeddings.tolist())
names
# dataset.class_to_idx

['IU',
 'IU',
 'IU',
 'IU',
 'IU',
 'chanyoung',
 'chanyoung',
 'chanyoung',
 'chanyoung',
 'chanyoung',
 'eunwoo',
 'eunwoo',
 'eunwoo',
 'eunwoo',
 'eunwoo',
 'gaeri',
 'gaeri',
 'gaeri',
 'gaeri',
 'gaeri',
 'hayoung',
 'hayoung',
 'hayoung',
 'hayoung',
 'hayoung',
 'jaewon',
 'jaewon',
 'jaewon',
 'jaewon',
 'jaewon',
 'jonggook',
 'jonggook',
 'jonggook',
 'jonggook',
 'jonggook',
 'min',
 'min',
 'min',
 'min',
 'min',
 'soohyang',
 'soohyang',
 'soohyang',
 'soohyang',
 'soohyang',
 'younha',
 'younha',
 'younha',
 'younha',
 'younha']

In [15]:
idxs = []
for n in names:
    idxs.append(dataset.class_to_idx[n])

In [16]:
from sklearn import svm

In [17]:
clf = svm.SVC(kernel='linear', probability=True)
clf.fit(embeddings.tolist(), idxs)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=True, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

### Test

In [18]:
data_dir = './data/test'

batch_size = 8
epochs = 16
workers = 0 if os.name == 'nt' else 4

In [19]:
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709,
    device=device
)

In [20]:
resnet = InceptionResnetV1(pretrained='vggface2', classify=True).eval().to(device)

In [21]:
def collate_fn(x):
    return x[0]

dataset = datasets.ImageFolder(data_dir)
dataset.idx_to_class = {i:c for c, i in dataset.class_to_idx.items()}
loader = DataLoader(dataset, collate_fn=collate_fn, num_workers=workers)

In [22]:
aligned = []
names = []
for x, y in loader:
    x_aligned, prob = mtcnn(x, return_prob=True)
    if x_aligned is not None:
        aligned.append(x_aligned)
        names.append(dataset.idx_to_class[y])

In [23]:
aligned = torch.stack(aligned).to(device)
embeddings = resnet(aligned).detach().cpu()

In [24]:
for i in clf.predict_proba(embeddings.tolist()):
    print(max(i))
    print(np.argmax(i))

0.3625982223694203
0
0.25168432557900555
0
0.4599965191660756
1
0.4245112873718769
1
0.4018164894846391
2
0.42744395317763395
2
0.4564392326777188
3
0.46992225780380786
3
0.4854342258980398
4
0.4796724560803571
4
0.41338187123350406
5
0.32202087968043036
5
0.4491405007483866
6
0.46899415159777824
6
0.3701196226338441
7
0.462430807751722
7
0.30815520424158577
8
0.4091222550773133
8
0.18853723016352036
0
0.21491975480721068
0


마지막 결과는 확률 상 가능 높은 수치 값과 index를 의미하는데,

마지막 데이터셋은 일부러 오답을 넣어두었다. 윤하 폴더에 전소미 사진을 넣어두었다.

전소미는 그냥 한국인인데 외국인 느낌이 나서 섭외해봤다.

개인적인 호감은 없다.

데이터를 보면 알 수 있듯이, 가장 높은 확률이 30 ~ 50 사이로 큰 차이로 형성되는 것을 볼 수 있다.

가장 높은 확률을 정답으로 택하면 모두 맞히지만, 학습되지 않은 얼굴인 전소미에 대해서 21프로의 결과를 얻은 것을 보았을 때, 25프로 아래는 unknown으로 처리해야 할 수준이다.